# Main Execution File

In [1]:
%pip install tensorflow
%pip install tensorflow.keras.models
%pip install tensorflow.keras.layers
%pip install scikeras

  Using cached keras-2.15.0-py3-none-any.whl.metadata (2.4 kB)
Using cached keras-2.15.0-py3-none-any.whl (1.7 MB)
  Attempting uninstall: keras
    Found existing installation: keras 3.8.0
    Uninstalling keras-3.8.0:
      Successfully uninstalled keras-3.8.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
scikeras 0.13.0 requires keras>=3.2.0, but you have keras 2.15.0 which is incompatible.

[notice] A new release of pip is available: 24.1.1 -> 25.0
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.
ERROR: Could not find a version that satisfies the requirement tensorflow.keras.models (from versions: none)
ERROR: No matching distribution found for tensorflow.keras.models

[notice] A new release of pip is available: 24.1.1 -> 25.0
[notice] To update, run: pip install --upgrade pip
Note: you m

In [2]:
import matplotlib.pyplot as plt, numpy as np, pandas as pd, seaborn as sb

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

## Stock Forecasting
Firstly, the prediction i am trying to make is Stock Forecasting with data (Jan 2024 - Jan 2025) to ensure the manufacurer has adequate stock for dispatch in time for the next order.

#### Notes
This would require doing some demand forecasting also (based on previous orders)? More easily done with reoccurant customers who order the same quantity of labels or similar quantities.

SP Inv [REL] = RELEASE QUANTITY - Why are some negative, is it a canceled shipment? (negative stock is what goes back to CAT1?)

In [3]:
# convert datasets to csv from txt

# \t used as separator, because of raw data format and the headers as row 0
products_s = pd.read_csv('./datasets/stock_forecasting/2022-2025/[LT] Products [STOCK].txt', sep='\t', header=0) 
sp_inventory_a = pd.read_csv('./datasets/stock_forecasting/2022-2025/[LT] SP Inventory [ADDS].txt', sep='\t', header=0)
sp_inventory_r = pd.read_csv('./datasets/stock_forecasting/2022-2025/[LT] SP Inventory [REL].txt', sep='\t', header=0)
tickets_c_i = pd.read_csv('./datasets/stock_forecasting/2022-2025/[LT] Tickets [CUSTOM] [ITEMS].txt', sep='\t', header=0)
tickets_c_m = pd.read_csv('./datasets/stock_forecasting/2022-2025/[LT] Tickets [CUSTOM] [MAIN].txt', sep='\t', header=0)

/var/folders/31/9bd8ksys1rzbjk8qs9scl95h0000gn/T/ipykernel_84417/449238093.py:5: DtypeWarning: Columns (14) have mixed types. Specify dtype option on import or set low_memory=False.
  sp_inventory_a = pd.read_csv('./datasets/stock_forecasting/2022-2025/[LT] SP Inventory [ADDS].txt', sep='\t', header=0)
/var/folders/31/9bd8ksys1rzbjk8qs9scl95h0000gn/T/ipykernel_84417/449238093.py:7: DtypeWarning: Columns (5,29,42,48) have mixed types. Specify dtype option on import or set low_memory=False.
  tickets_c_i = pd.read_csv('./datasets/stock_forecasting/2022-2025/[LT] Tickets [CUSTOM] [ITEMS].txt', sep='\t', header=0)


In [4]:
# # convert 'Entered_Date' to datetime format
# sp_inventory_a['Entered_Date'] = pd.to_datetime(sp_inventory_a['Entered_Date'], format='%m/%d/%y')
# sp_inventory_r['Entered_Date'] = pd.to_datetime(sp_inventory_r['Entered_Date'], format='%m/%d/%y')

# # limit the dates to 2024-2025
# sp_inventory_a = sp_inventory_a[(sp_inventory_a['Entered_Date'] >= '2024-01-01') & (sp_inventory_a['Entered_Date'] <= '2025-01-01')]
# sp_inventory_r = sp_inventory_r[(sp_inventory_r['Entered_Date'] >= '2024-01-01') & (sp_inventory_r['Entered_Date'] <= '2025-01-01')]

# # display row number amount
# print('SP Inventory Adds: ', sp_inventory_a.shape)
# print('SP Inventory Rel: ', sp_inventory_r.shape)

In [5]:
# output first 5 rows of each dataset
pd.set_option('display.max_columns', None)

# change column name of 'ProductNo' to 'ProductNumber'
products_s.rename(columns={'ProductNo': 'ProductNumber'}, inplace=True)

print('PRODUCTS STOCK\n', products_s.head())
print('PRODUCTS STOCK COLS\n', products_s.columns)
print('----------------------------\n')

PRODUCTS STOCK
   Adhesive,Alternate,Available,BackOrdered,Box_Size,CaseQty,Color,Commission,Cost,Currency_ExchangeRate,Currency_ID,Customer_Num,CustomerName,Desc1,Desc2,EnteredDate,EnteredTime,eTraxx_Forecast_Quantity,eTraxx_Forecast_Range,FC_Cost,File_Name,ID,Inactive,InternetQuery,Inventory_Expires,Is_LinkedCustomPricesUsed,Is_Location_UsedBy_Consignment,Link_Factor,Location,Material,MaxProduce,MinProduce,ModifiedDate,ModifiedTime,newTimeDateStamp,OnOrder,PackageQty,Part_Type,PhysicalInv,PictDisplayFormat,PK_UUID,PriceMode,PriceMode_Local,ProdClass,ProdSubClass,Product_Image_Size,Product_UniqueProdID,Production_Waste,ProductNo,Releases_Allowed_Default,SupplierName,SupplierNo,SupplierNotes,SupplierPartNo,Tag,TotalCost,UPC,Updated,updateTimeDateStamp,Weight
0  ,,0,0,,,,0,0.0,0,0,1.0,Catapult,,,06/07/18,09:...                                                                                                                                                                                   

In [6]:
print('SALES PRICE INV ADD\n', sp_inventory_a.head())
print('SALES PRICE INV ADD COLS\n', sp_inventory_a.columns)
print('----------------------------\n')

SALES PRICE INV ADD
    ID  StockProduct_ID Entered_Date Entered_Time       Entered_By  \
0   1                3     06/16/18     04:46:02   Andrew Burgess   
1   2                2     07/10/18     12:13:25   Andrew Burgess   
2   3                2     07/10/18     12:19:05   Andrew Burgess   
3   4                6     10/11/18     17:43:03       Lewis Cook   
4   5                7     10/12/18     09:48:14  Charlie Ellison   

  Inventory_Date Inventory_Time  Inventory_DateTime_Stamp Expiration_Date  \
0       06/16/18       04:46:02                 897972362        00/00/00   
1       07/10/18       12:13:25                 900072805        00/00/00   
2       07/10/18       12:19:05                 900073145        00/00/00   
3       10/11/18       17:43:03                 908127783        00/00/00   
4       10/12/18       09:48:14                 908185694        00/00/00   

   Inventory_Quantity  Inventory_Balance Cost_Unit  Item_Cost Location  \
0              150000      

In [7]:
print('SALES PRICE INV REL\n', sp_inventory_r.head())
print('SALES PRICE INV REL COLS\n', sp_inventory_r.columns)
print('----------------------------\n')

SALES PRICE INV REL
   Cost_Unit Cost_Unit_Local       Entered_By Entered_Date Entered_Time  ID  \
0      Each            Each   Andrew Burgess     06/16/18     06:02:09   1   
1      Each            Each   Andrew Burgess     07/10/18     12:18:24   2   
2      Each            Each  Charlie Ellison     10/12/18     10:04:21   3   
3      Each            Each  Charlie Ellison     10/24/18     13:34:47   4   
4      Each            Each  Charlie Ellison     10/24/18     13:35:18   5   

   Is_User_Made  Item_Cost  Kit_Part_Number  Kit_Product_ID Location  \
0         False        0.0              NaN             NaN      NaN   
1         False        0.0              NaN             NaN      NaN   
2         False        0.0              NaN             NaN      NaN   
3         False        0.0              NaN             NaN      NaN   
4         False        0.0              NaN             NaN      NaN   

   Modification_Count Modified_By Modified_Date Modified_Time  \
0           

In [8]:
print('TICKETS CUSTOM ITEMS\n', tickets_c_i.head())
print('TICKETS CUSTOM ITEMS COLS\n', tickets_c_i.columns)
print('----------------------------\n')

TICKETS CUSTOM ITEMS
      Art  Art_Item  Art_Ticket  Assigned                        ColorDescr  \
0  False     False       False       NaN         C: WHITE, CMYOK, LAM ADH    
1  False     False       False       NaN              C: MYK, GLOSS 54121    
2  False     False       False       NaN  C: CMYK,7512C,363C,286C,LAM ADH    
3  False     False       False       NaN  C: CMYK,7512C,363C,286C,LAM ADH    
4  False     False       False       NaN                C: CMYK, GLOSS V     

  ConsecNo  CostM  Desc2                                   Description  \
0      NaN      0    NaN            510016W Gumout All In One FSC 10oz   
1      NaN      0    NaN       510022W Gumout Octane Booster 10oz Wrap   
2      NaN      0    NaN   Simply Nature 93/7 Ground Beef Famil Pack B   
3      NaN      0    NaN  Simply Nature 93/7 Ground Beef Family Pack F   
4      NaN      0    NaN      72969 FL Ginger Sweet Chili Sauce 14.5oz   

   ediLineNumber  Equip_NoColors  Equip_NoFloods  Equip_Null_Cyc

In [9]:
# change the column name of 'Number' to 'TicketNumber'
tickets_c_m.rename(columns={'Number': 'TicketNumber'}, inplace=True)

print('TICKETS CUSTOM MAIN\n', tickets_c_m.head())
print('TICKETS CUSTOM MAIN COLS\n', tickets_c_m.columns)
print('----------------------------\n')

TICKETS CUSTOM MAIN
   Act_MakeReady_Footage,Act_OTHER_Hours,ActArtwork,ActFinMaterial,ActFootage,ActMRHrs,ActPackHrs,ActPostPressHours,ActPressSpd,ActQuantity,ActRunHrs,ActStockCost,ActTotalCost,ActualBillings_NetOfSalesTax,ActualCommissionsCost,ActualFanFoldCost,ActualFanfoldHours,ActualFanfoldRate,ActualFootage_StockRolls,ActualGrossMargin_Dollars,ActualGrossMargin_Percent,ActualMSI_StockRolls,ActualNumOfStockRolls,ActualOtherLaborCost,ActualPackagingRate,ActualPackingLaborCost,ActualPostPressLaborCost,ActualPressCost,ActualPressHours,ActualPressRate,ActualRewindingCost,ActualRewindingHours,ActualRewindingRate,ActualTotalFinishing,ActualTotalHours,ActualTotalLaborCosts,ActualTotalMatAndFreightCost,ActualTotalPOCosts,ActWuHrs,AmortizeColorChanges,AmortizePlateChanges,Are_Tools_for_Equip,ArtDone,ArtStat,AutoAppl,BackStage_ColorStrategy,BackStage_DefaultReportForm,BackStage_SmartMarkSet,Bill_Address_ID,Bill_TaxRegion_ID,BillAddr1,BillAddr2,BillCity,BillCountry,BillCounty,BillLocation,B

In [10]:
# print('TICKETS STOCK ITEMS\n', tickets_s_i.head())
# print('TICKETS STOCK ITEMS COLS\n', tickets_s_i.columns)

In [11]:
# # change the column name of 'Number' to 'TicketNumber'
# tickets_s_m.rename(columns={'Number': 'TicketNumber'}, inplace=True)

# print('TICKETS STOCK MAIN\n', tickets_s_m.head())
# print('TICKETS STOCK MAIN COLS\n', tickets_s_m.columns)

In [12]:
# check for missing values
print('PRODUCTS STOCK\n' ,products_s.isnull().sum())
print('----------------------------\n')
print('SALES PRICE INV ADD\n', sp_inventory_a.isnull().sum())
print('----------------------------\n')
print('SALES PRICE INV REL\n', sp_inventory_r.isnull().sum())
print('----------------------------\n')
print('TICKETS CUSTOM ITEMS\n', tickets_c_i.isnull().sum())
print('----------------------------\n')
print('TICKETS CUSTOM MAIN\n', tickets_c_m.isnull().sum())
print('----------------------------\n')

PRODUCTS STOCK
 Adhesive,Alternate,Available,BackOrdered,Box_Size,CaseQty,Color,Commission,Cost,Currency_ExchangeRate,Currency_ID,Customer_Num,CustomerName,Desc1,Desc2,EnteredDate,EnteredTime,eTraxx_Forecast_Quantity,eTraxx_Forecast_Range,FC_Cost,File_Name,ID,Inactive,InternetQuery,Inventory_Expires,Is_LinkedCustomPricesUsed,Is_Location_UsedBy_Consignment,Link_Factor,Location,Material,MaxProduce,MinProduce,ModifiedDate,ModifiedTime,newTimeDateStamp,OnOrder,PackageQty,Part_Type,PhysicalInv,PictDisplayFormat,PK_UUID,PriceMode,PriceMode_Local,ProdClass,ProdSubClass,Product_Image_Size,Product_UniqueProdID,Production_Waste,ProductNo,Releases_Allowed_Default,SupplierName,SupplierNo,SupplierNotes,SupplierPartNo,Tag,TotalCost,UPC,Updated,updateTimeDateStamp,Weight    0
dtype: int64
----------------------------

SALES PRICE INV ADD
 ID                                 0
StockProduct_ID                    0
Entered_Date                       0
Entered_Time                       0
Entered_By      

In [13]:
# drop columns with all rows as missing values (NaN/0)
columns_to_drop = products_s.columns[products_s.isnull().all()].tolist()
print(f'Products STOCK - Columns to be dropped: {columns_to_drop}\n')
products_s = products_s.dropna(axis=1, how='all')

# set the name attribute for each DataFrame
products_s.name = 'Products STOCK'
sp_inventory_a.name = 'SP Inventory Adds'
sp_inventory_r.name = 'SP Inventory Rel'
tickets_c_m.name = 'Tickets CUSTOM MAIN'

# drop columns with all rows containing the same values of either 0, True, or False
for df in [products_s, sp_inventory_a, sp_inventory_r, tickets_c_m]:
    columns_to_drop = [col for col in df.columns if df[col].nunique() == 1 and df[col].iloc[0] in [0, True, False]]
    print(f'{df.name} - Columns to be dropped: {columns_to_drop}\n')
    df.drop(columns=columns_to_drop, inplace=True)

columns_to_drop = tickets_c_m.columns[tickets_c_m.isnull().all()].tolist()
print(f'Tickets CUSTOM MAIN - Columns to be dropped: {columns_to_drop}\n')
tickets_c_m = tickets_c_m.dropna(axis=1, how='all')


Products STOCK - Columns to be dropped: []

Products STOCK - Columns to be dropped: []

SP Inventory Adds - Columns to be dropped: []

SP Inventory Rel - Columns to be dropped: ['Modification_Count']

Tickets CUSTOM MAIN - Columns to be dropped: []

Tickets CUSTOM MAIN - Columns to be dropped: []



In [14]:
print('TICKETS CUSTOM MAIN\n', tickets_c_m.head())
print('TICKETS CUSTOM MAIN COLS\n', tickets_c_m.columns)
print('----------------------------\n')

In [15]:
# check which files have the column 'ProductNumber' in common
dataframes = [products_s, sp_inventory_a, sp_inventory_r, tickets_c_i, tickets_c_m]
common_column = 'ProductNumber'
po_column = 'PONumber'

# Set the name attribute for each DataFrame if not already set
for df, name in zip(dataframes, ['Products STOCK', 'SP Inventory Adds', 'SP Inventory Rel', 'Tickets CUSTOM ITEMS', 'Tickets CUSTOM MAIN']):
	if not hasattr(df, 'name'):
		df.name = name

# Check which dataframes contain the common column
for df in dataframes:
	if common_column in df.columns:
		print(f"{df.name} contains the column '{common_column}'")
	else:
		continue

Tickets CUSTOM ITEMS contains the column 'ProductNumber'


In [16]:
# Specify the columns to display along with 'TicketNumber'
columns_to_display = ['TicketNumber']

# Find common 'TicketNumber' values
common_ticket_numbers = set(tickets_c_m['TicketNumber']).intersection(tickets_c_i['TicketNumber'])

# Filter rows with common 'TicketNumber' values and select specific columns
common_tickets_c_m = tickets_c_m[tickets_c_m['TicketNumber'].isin(common_ticket_numbers)][columns_to_display + ['TicQuantity', 'OrderDate', 'Ship_by_Date']]
common_tickets_c_i = tickets_c_i[tickets_c_i['TicketNumber'].isin(common_ticket_numbers)][columns_to_display + ['OrderQuantity', 'ProductNumber', 'Description']]


KeyError: 'TicketNumber'

In [ ]:
# main job description area (summaritive information)
print('Common Tickets CUSTOM MAIN\n', common_tickets_c_m.head())
print('----------------------------\n')

In [ ]:
# job details area (detailed information)
print('Common Tickets CUSTOM ITEMS\n', common_tickets_c_i.head())
print('----------------------------\n')

In [20]:
# Specify the columns to display along with 'ProductNumber'
columns_to_display = ['ProductNumber']

# Find common 'TicketNumber' values
common_ticket_numbers = set(products_s['ProductNumber']).intersection(tickets_c_i['ProductNumber'])

# Filter rows with common 'TicketNumber' values and select specific columns
common_products_s = products_s[products_s['ProductNumber'].isin(common_ticket_numbers)][columns_to_display + ['PhysicalInv', 'Location']]

In [ ]:
# product details related to tickets custom items (link between products and tickets)
print('Common Products STOCK\n', common_products_s.head())
print('----------------------------\n')

In [22]:
# Merge common_tickets_c_m and common_tickets_c_i on 'TicketNumber'
merged_tickets = pd.merge(common_tickets_c_m, common_tickets_c_i, on='TicketNumber', how='inner')

# Ticket MAIN/ITEMS + Products STOCK -> Merged DataFrame
# Merge the result with common_products_s on 'ProductNumber'
stock_forecasting_merge_df = pd.merge(merged_tickets, common_products_s, on='ProductNumber', how='inner')

In [ ]:
'''
REASON FOR 'TicQuantity' AND 'OrderQuantity' DIFFERENCE:

Ticket Quantity (TicQuantity) [MAIN]: Amount of labels ordered by the customer (multiple different products can be ordered in one ticket)

Order Quantity (OrderQuantity) [ITEMS]: Amount of the same product ordered by the customer (one product)
'''

# check if all 'TicQuantity' values are the same as 'OrderQuantity', output boolean
print('TicQuantity == OrderQuantity\n', stock_forecasting_merge_df['TicQuantity'].eq(stock_forecasting_merge_df['OrderQuantity']).all())
print('----------------------------\n')

# output all 'orderQuantity' values from highest to lowest
print('OrderQuantity\n', stock_forecasting_merge_df['OrderQuantity'].sort_values(ascending=False).head(20))
print('----------------------------\n')

#### 'TicQuantity' is good for total ticket quantity, 'OrderQuantity' is good for individual product quantity

Dropping this keeps the project from being confusing. It works without this column since there are multiple 'TicketNumber' values with the item specific 'OrderQuantity'


In [24]:
stock_forecasting_merge_df = stock_forecasting_merge_df.drop(columns=['TicQuantity'])

In [ ]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error
from sklearn.preprocessing import LabelEncoder

# Merge datasets
merged_data = pd.merge(tickets_c_m, tickets_c_i, on='TicketNumber')
merged_data = pd.merge(merged_data, products_s, on='ProductNumber')

# Feature engineering
merged_data['OrderDate'] = pd.to_datetime(merged_data['OrderDate'])
merged_data['Ship_by_Date'] = pd.to_datetime(merged_data['Ship_by_Date'])
merged_data['order_month'] = merged_data['OrderDate'].dt.month
merged_data['order_week'] = merged_data['OrderDate'].dt.isocalendar().week

# Aggregate data
product_sales = merged_data.groupby(['ProductNumber', 'order_month']).agg({'OrderQuantity': 'sum'}).reset_index()

# Encode 'ProductNumber' as numeric
label_encoder = LabelEncoder()
product_sales['ProductNumber'] = label_encoder.fit_transform(product_sales['ProductNumber'])

# Define features and target
X = product_sales[['ProductNumber', 'order_month']]
y = product_sales['OrderQuantity']

# Split data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle=False)

# Train model
model = RandomForestRegressor()
model.fit(X_train, y_train)

# Evaluate model
y_pred = model.predict(X_test)
mae = mean_absolute_error(y_test, y_pred)
print(f'Mean Absolute Error: {mae}')

In [ ]:
# edit the structure of the merged DataFrame
stock_forecasting_merge_df = stock_forecasting_merge_df[
    ['TicketNumber',
     'OrderDate', 
     'Ship_by_Date',
     'PhysicalInv',
     'OrderQuantity', 
     'ProductNumber', 
     'Description',
     'Location']]

# Display the merged DataFrame
print('Final Merged DataFrame\n', stock_forecasting_merge_df.head(60))
print('----------------------------\n')

In [ ]:
# save all rows with values of 'PhysicalInv' as 0 into one DataFrame
zero_physical_inv = stock_forecasting_merge_df[stock_forecasting_merge_df['PhysicalInv'] == 0]

# save all rows with values of 'PhysicalInv' > 0 into one DataFrame
positive_physical_inv = stock_forecasting_merge_df[stock_forecasting_merge_df['PhysicalInv'] > 0]

print('Zero Physical Inventory\n', zero_physical_inv.head())
print('----------------------------\n')

print('Positive Physical Inventory\n', positive_physical_inv.head())
print('----------------------------\n')

In [ ]:
# check if zero_physical_inv has 'OrderQuantity' any values of 0
print('OrderQuantity == 0\n', zero_physical_inv['OrderQuantity'].eq(0).any())
print('----------------------------\n')

In [ ]:
# print the amount of rows in each DataFrame
print('Zero Physical Inventory\n', zero_physical_inv.shape)
print('----------------------------\n')

# drop values that are both order quantity and physical inventory are 0
zero_physical_inv = zero_physical_inv[~((zero_physical_inv['PhysicalInv'] == 0) & (zero_physical_inv['OrderQuantity'] == 0))]

# check if zero_physical_inv has 'OrderQuantity' any values of 0
print('OrderQuantity == 0\n', zero_physical_inv['OrderQuantity'].eq(0).any())
print('----------------------------\n')

# print the amount of rows in each DataFrame
print('Zero Physical Inventory\n', zero_physical_inv.shape)
print('----------------------------\n')

In [ ]:
# display the first rows of zero_physical_inv
print('Zero Physical Inventory\n', zero_physical_inv.head(20))

In [ ]:
# display the first rows of positive_physical_inv
print('Positive Physical Inventory\n', positive_physical_inv.head(20))

In [ ]:
# print the amount of rows in each DataFrame
print('Positive Physical Inventory\n', positive_physical_inv.shape)
print('----------------------------\n')

positive_physical_inv = positive_physical_inv[positive_physical_inv['OrderQuantity'] > 0]

# check if positive_physical_inv has 'OrderQuantity' any values of 0
print('OrderQuantity == 0\n', positive_physical_inv['OrderQuantity'].eq(0).any())
print('----------------------------\n')

# print the amount of rows in each DataFrame
print('Positive Physical Inventory\n', positive_physical_inv.shape)
print('----------------------------\n')

In [ ]:
# display the first rows of positive_physical_inv
print('Positive Physical Inventory\n', positive_physical_inv.head(20))
print('----------------------------\n')

# display the first rows of zero_physical_inv
print('Zero Physical Inventory\n', zero_physical_inv.head(20))
print('----------------------------\n')

In [34]:
# create a method to find the best possible hyperparameters for the model
def find_best_hyperparameters(model, parameter_grid, X_train, y_train):
    from sklearn.model_selection import GridSearchCV

    grid_search = GridSearchCV(estimator=model, param_grid=parameter_grid, cv=5, n_jobs=-1, verbose=2)
    grid_search.fit(X_train, y_train)

    print(f'{model.__class__.__name__} Best Parameters: {grid_search.best_params_}')    

    return grid_search.best_params_

In [35]:
from sklearn.model_selection import cross_val_predict
from sklearn.metrics import mean_absolute_error, mean_squared_error
import seaborn as sns
from sklearn.metrics import r2_score

def evaluate_model(model, X, y):
    print('Evaluating model...')
    
    # Output type of model
    print(f"Model: {model.__class__.__name__}")

    # K-fold cross validation (k=5)
    y_pred = cross_val_predict(model, X, y, cv=5)

    # Mean Absolute Error (MAE)
    mae = mean_absolute_error(y, y_pred)
    print(f"Mean Absolute Error (MAE): {mae:.4f}")

    # Mean Squared Error (MSE)
    mse = mean_squared_error(y, y_pred)
    print(f"Mean Squared Error (MSE): {mse:.4f}")

    # Root Mean Squared Error (RMSE)
    rmse = np.sqrt(mse)
    print(f"Root Mean Squared Error (RMSE): {rmse:.4f}")

    # R-squared (R²)
    r2 = r2_score(y, y_pred)
    print(f"R-squared (R²): {r2:.4f}")


In [ ]:
# Group by 'ProductNumber' and create separate arrays for 'OrderQuantity'
product_groups = stock_forecasting_merge_df.groupby('ProductNumber')['OrderQuantity'].apply(list)

# Convert the grouped data into a dictionary of arrays
product_order_quantities = {product: np.array(quantities) for product, quantities in product_groups.items()}

# Filter the dictionary to only include product numbers with at least one order quantity over 300,000 (edit the threshold as needed to find out for feature engineering)
filtered_product_order_quantities = {product: quantities for product, quantities in product_order_quantities.items() if any(quantities > 300000)}

# Display the filtered arrays
print('Filtered Product Order Quantities (300k for one Order Quantity)\n')
for product, quantities in filtered_product_order_quantities.items():
    print(f'Product: {product}, Order Quantities: {quantities}')
    print('----------------------------\n')

In [ ]:
from sklearn.preprocessing import StandardScaler

# Ensure 'OrderDate' and 'Ship_by_Date' are in datetime format
stock_forecasting_merge_df['OrderDate'] = pd.to_datetime(stock_forecasting_merge_df['OrderDate'])

# Handle invalid date values in 'Ship_by_Date'
stock_forecasting_merge_df['Ship_by_Date'] = pd.to_datetime(
    stock_forecasting_merge_df['Ship_by_Date'], errors='coerce'
)

# Calculate 'Lead_Time' in days
stock_forecasting_merge_df['Lead_Time'] = (stock_forecasting_merge_df['Ship_by_Date'] - stock_forecasting_merge_df['OrderDate']).dt.days
# Handle missing values in 'Lead_Time'
stock_forecasting_merge_df['Lead_Time'].fillna(stock_forecasting_merge_df['Lead_Time'].median(), inplace=True)

# Convert 'OrderQuantity' to numeric, forcing errors to NaN and then filling them with 0
stock_forecasting_merge_df['OrderQuantity'] = pd.to_numeric(stock_forecasting_merge_df['OrderQuantity'], errors='coerce').fillna(0)

# define the features and target variable
X = stock_forecasting_merge_df[['PhysicalInv', 'Lead_Time']]
y = stock_forecasting_merge_df['OrderQuantity']

# Normalize the features
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Train-test split with scaled features
X_train_scaled, X_val_scaled, y_train, y_val = train_test_split(X_scaled, y, test_size=0.2, random_state=42)

## Make model using Positive PhysicalInv dataset

In [ ]:
# Ensure 'OrderDate' and 'Ship_by_Date' are in datetime format
positive_physical_inv['OrderDate'] = pd.to_datetime(positive_physical_inv['OrderDate'])

# Handle invalid date values in 'Ship_by_Date'
positive_physical_inv['Ship_by_Date'] = pd.to_datetime(
    positive_physical_inv['Ship_by_Date'], errors='coerce'
)

# Calculate 'Lead_Time' in days
positive_physical_inv['Lead_Time'] = (positive_physical_inv['Ship_by_Date'] - positive_physical_inv['OrderDate']).dt.days
# Handle missing values in 'Lead_Time'
positive_physical_inv['Lead_Time'].fillna(positive_physical_inv['Lead_Time'].median(), inplace=True)

# Convert 'OrderQuantity' to numeric, forcing errors to NaN and then filling them with 0
positive_physical_inv['OrderQuantity'] = pd.to_numeric(positive_physical_inv['OrderQuantity'], errors='coerce').fillna(0)

# define the features and target variable
X = positive_physical_inv[['PhysicalInv', 'Lead_Time']]
y = positive_physical_inv['OrderQuantity']

# Normalize the features
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Train-test split with scaled features
X_train_scaled, X_val_scaled, y_train, y_val = train_test_split(X_scaled, y, test_size=0.2, random_state=42)

In [39]:
'''
--------------------------------------------------------
REGRESSION MODEL PARAM GRIDS
--------------------------------------------------------
'''
rf_param_grid = { # Random Forest Regressor
    'n_estimators': [100, 200, 300],
    'max_depth': [None, 5, 10, 15],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4],
    'criterion': ['squared_error'], # tested other values, but this was the best one
    'n_jobs': [1, 2, 4],
    'random_state': [42]
}

lr_param_grid = { # Linear Regression
    'fit_intercept': [True, False],
    'positive': [True, False],
    'n_jobs': [1, 2, 4],
    'copy_X': [True, False]
}

svr_param_grid = { # Support Vector Regressor
    'kernel': ['linear', 'rbf', 'sigmoid'], # 'poly' takes long to compute
    'C': [0.1, 1, 10, 100],
    'gamma': ['scale', 'auto']
}

dt_param_grid = { # Decision Tree Regressor
    'max_depth': [None, 5, 10, 15],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4],
    'random_state': [42],
    'splitter': ['best', 'random']
}

nn_param_grid = { # Neural Network Regressor
    'model__epochs': [50, 100],
    'model__batch_size': [32, 64],
}

'''
--------------------------------------------------------
CLASSIFICATION MODELS
--------------------------------------------------------
'''

rf_class_param_grid = { # Random Forest Classifier
    'n_estimators': [100, 200, 300],
    'max_depth': [None, 5, 10, 15],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4],
    'criterion': ['gini'],
    'n_jobs': [1, 2, 4],
    'random_state': [42]
}

lr_class_param_grid = { # Logistic Regression
    'penalty': ['l1', 'l2', 'elasticnet', 'none'],
    'C': [0.1, 1, 10, 100],
    'solver': ['newton-cg', 'lbfgs', 'liblinear', 'sag', 'saga'],
    'max_iter': [100, 200, 300],
    'n_jobs': [1, 2, 4]
}

svc_class_param_grid = { # Support Vector Classifier
    'kernel': ['rbf', 'sigmoid'],
    'C': [0.1, 1, 10, 100],
    'gamma': ['scale', 'auto'],
}

dt_class_param_grid = { # Decision Tree Classifier
    'max_depth': [None, 5, 10, 15],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4],
    'random_state': [42],
    'splitter': ['best', 'random']
}    

In [40]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, classification_report
import seaborn as sns
from seaborn import heatmap

# Define the features and target variable for classification
X_class = stock_forecasting_merge_df[['PhysicalInv', 'Lead_Time']]
y_class = stock_forecasting_merge_df['OrderQuantity'].apply(lambda x: 1 if x > 0 else 0)  # Binary classification: 1 if OrderQuantity > 0, else 0

# Train-test split for classification
X_train_class, X_test_class, y_train_class, y_test_class = train_test_split(X_class, y_class, test_size=0.2, random_state=42)

In [ ]:
# Train the classification model (Random Forest Classifier) with param grid
rf_classifier = RandomForestClassifier()
rf_class_params = find_best_hyperparameters(rf_classifier, parameter_grid=rf_class_param_grid, X_train=X_train_class, y_train=y_train_class)

# Train the classification model with the best hyperparameters
rf_classifier = RandomForestClassifier(**rf_class_params)
rf_classifier.fit(X_train_class, y_train_class)

# Predict using the classification model
y_pred_class = rf_classifier.predict(X_test_class)

In [ ]:
from sklearn.linear_model import LogisticRegression

# Train the classification model (Logistic Regression) with param grid
lr_classifier = LogisticRegression()
lr_class_params = find_best_hyperparameters(lr_classifier, parameter_grid=lr_class_param_grid, X_train=X_train_class, y_train=y_train_class)

# Train the classification model with the best hyperparameters
lr_classifier = LogisticRegression(**lr_class_params)
lr_classifier.fit(X_train_class, y_train_class)

# Predict using the classification model
y_pred_class = lr_classifier.predict(X_test_class)

In [ ]:
# support vector classifier
from sklearn.svm import SVC

# Train the classification model (Support Vector Classifier) with param grid
svc_classifier = SVC()
svc_class_params = find_best_hyperparameters(svc_classifier, parameter_grid=svc_class_param_grid, X_train=X_train_class, y_train=y_train_class)

# Train the classification model with the best hyperparameters
svc_classifier = SVC(**svc_class_params)
svc_classifier.fit(X_train_class, y_train_class)

# Predict using the classification model
y_pred_class = svc_classifier.predict(X_test_class)

In [ ]:
# decision tree classifier
from sklearn.tree import DecisionTreeClassifier

# Train the classification model (Decision Tree Classifier) with param grid
dt_classifier = DecisionTreeClassifier()
dt_class_params = find_best_hyperparameters(dt_classifier, parameter_grid=dt_class_param_grid, X_train=X_train_class, y_train=y_train_class)

# Train the classification model with the best hyperparameters
dt_classifier = DecisionTreeClassifier(**dt_class_params)
dt_classifier.fit(X_train_class, y_train_class)

# Predict using the classification model
y_pred_class = dt_classifier.predict(X_test_class)

In [ ]:
# Plot the confusion matrix
conf_matrix = confusion_matrix(y_test_class, y_pred_class)
plt.figure(figsize=(10, 7))
sns.heatmap(conf_matrix, annot=True, fmt='d', cmap='Blues', xticklabels=['Predicted 0', 'Predicted 1'], yticklabels=['Actual 0', 'Actual 1'])
plt.xlabel('Predicted')
plt.ylabel('Actual')
plt.title('Confusion Matrix')
plt.show()

# Display the classification report
class_report = classification_report(y_test_class, y_pred_class)
print('Classification Report\n', class_report)

### Model Training - Takes about 40 minutes for all models (try not to keep running)

In [ ]:
from sklearn.ensemble import RandomForestRegressor

# Train the model with scaled features (Random Forest Regressor)
rf = RandomForestRegressor()
rf_params = find_best_hyperparameters(rf, rf_param_grid, X_train_scaled, y_train)

In [42]:
rf = RandomForestRegressor(**rf_params)
rf.fit(X_train_scaled, y_train)
y_pred_rf = rf.predict(X_val_scaled)

In [ ]:
# Train the model with scaled features (Linear Regression)
lr = LinearRegression()
lr_params = find_best_hyperparameters(lr, lr_param_grid, X_train_scaled, y_train)

In [35]:
lr = LinearRegression(**lr_params)
lr.fit(X_train_scaled, y_train)
y_pred_lr = lr.predict(X_val_scaled)

In [ ]:
# Train the model with scaled features (Support Vector Regressor)
from sklearn.svm import SVR
svr = SVR()
svr_params = find_best_hyperparameters(svr, svr_param_grid, X_train_scaled, y_train)

In [36]:
svr = SVR(**svr_params)
svr.fit(X_train_scaled, y_train)
y_pred_svr = svr.predict(X_val_scaled)

In [ ]:
# Train the model with scaled features (Decision Tree Regressor)
from sklearn.tree import DecisionTreeRegressor
dt = DecisionTreeRegressor()
dt_params = find_best_hyperparameters(dt, dt_param_grid, X_train_scaled, y_train)

In [38]:
dt = DecisionTreeRegressor(**dt_params)
dt.fit(X_train_scaled, y_train)
y_pred_dt = dt.predict(X_val_scaled)

In [ ]:
# # NN METRICS ARE NOT ARENT BETTER THAN BASE MODELS, SO ITS COMMENTED OUT

# import tensorflow as tf
# from tensorflow.keras.models import Sequential
# from tensorflow.keras.layers import Dense
# from scikeras.wrappers import KerasRegressor
# from sklearn.model_selection import GridSearchCV, cross_val_score

# # Define the neural network model
# def create_nn_model(input_shape):
#     model = Sequential()
#     model.add(Dense(64, input_dim=input_shape, activation='relu'))
#     model.add(Dense(32, activation='relu'))
#     model.add(Dense(1, activation='linear'))
#     model.compile(optimizer='adam', loss='mean_squared_error', metrics=['mean_absolute_error'])
#     return model

# # Wrap the Keras model with KerasRegressor
# input_shape = X_train_scaled.shape[1]
# nn_model = KerasRegressor(build_fn=create_nn_model, input_shape=input_shape, verbose=1)

# # Define the parameter grid for GridSearchCV
# nn_param_grid = {
#     'epochs': [50, 100],
#     'batch_size': [32, 64]
# }

# # Perform GridSearchCV
# grid_search = GridSearchCV(estimator=nn_model, param_grid=nn_param_grid, cv=5, n_jobs=-1, verbose=2)
# grid_search.fit(X_train_scaled, y_train)

# # Get the best parameters
# nn_params = grid_search.best_params_
# print(f'Best Parameters: {nn_params}')

# # Set the best parameters to the model
# nn_model.set_params(**nn_params)

# # Use cross_val_score or other scikit-learn utilities
# scores = cross_val_score(nn_model, X_train_scaled, y_train, cv=5)
# print(scores)

# # Train the model
# history = nn_model.fit(X_train_scaled, y_train, epochs=nn_params['epochs'], batch_size=nn_params['batch_size'], validation_split=0.2, verbose=1)

# # Predict using the neural network model
# y_pred_nn = nn_model.predict(X_val_scaled)

# # Evaluate the model
# evaluate_model(nn_model, X_scaled, y)

In [ ]:
# # LSTM model COMMENTED OUT DUE TO METRICS NOT BEING IMPROVED

# from keras.layers import LSTM, Dropout
# from keras.preprocessing.sequence import TimeseriesGenerator

# # Define the LSTM model
# def create_lstm_model(input_shape):
#     model = Sequential()
#     model.add(LSTM(50, activation='relu', input_shape=input_shape))
#     model.add(Dropout(0.2))
#     model.add(Dense(1))
#     model.compile(optimizer='adam', loss='mean_squared_error', metrics=['mean_absolute_error'])
#     return model

# # Prepare the data for LSTM
# # Reshape the data to be 3D [samples, timesteps, features]
# X_reshaped = X_scaled.reshape((X_scaled.shape[0], 1, X_scaled.shape[1]))

# # Split the reshaped data into training and validation sets
# X_train_reshaped, X_val_reshaped, y_train, y_val = train_test_split(X_reshaped, y, test_size=0.2, random_state=42)

# # Create the LSTM model
# input_shape = (X_train_reshaped.shape[1], X_train_reshaped.shape[2])
# lstm_model = create_lstm_model(input_shape)

# # Train the LSTM model
# history = lstm_model.fit(X_train_reshaped, y_train, epochs=50, batch_size=32, validation_data=(X_val_reshaped, y_val), verbose=1)

# # Predict using the LSTM model
# y_pred_lstm = lstm_model.predict(X_val_reshaped)

# # Evaluate the LSTM model
# evaluate_model(lstm_model, X_reshaped, y)

In [ ]:
evaluate_model(rf, X_scaled, y)
print('--------------------------------------------')

In [ ]:
evaluate_model(lr, X_scaled, y)
print('--------------------------------------------')

In [ ]:
evaluate_model(svr, X_scaled, y)
print('--------------------------------------------')

In [ ]:
evaluate_model(dt, X_scaled, y)
print('--------------------------------------------')

Random Forest with CV & Outliter removal
| Evaluation Metric | Model |   
| --- | --- |
| MAE | 84860.9381 |
| MSE | 41863556821.9018 |
| RMSE | 204605.8573 |
| R2 | 0.1196 |

Random Forest with CV & Param grid (With outliers)
| Evaluation Metric | Model |   
| --- | --- |
| MAE | 81106.3853 |
| MSE | 29933938412.4621 |
| RMSE | 173014.2723 |
| R2 | 0.2059 |

Random Forest - CV & Outlier removal & Param grid (Doesnt look correct on plot)
| Evaluation Metric | Model |
| --- | --- |
| MAE | 7.2066 |
| MSE | 758.3109 |
| RMSE | 27.5374 |
| R2 | -0.1655 |

Random Forest - PhysicalInv > 0 (CV & Paramgrid)

| Evaluation Metric | Model |
| --- | --- |
| MAE | 104017.0708 |
| MSE | 47589838444.2892 |
| RMSE | 218150.9533 |
| R2 | 0.3368 |

In [ ]:
## ALL ENSEMBLE METHODS ARE COMMENTED OUT BECAUSE METRICS ARE NOT IMPROVED

# # implement ensemble methods (bagging, boosting, stacking) to improve the model
# from sklearn.ensemble import BaggingRegressor, AdaBoostRegressor, StackingRegressor
# from sklearn.linear_model import RidgeCV

# print('--------------------- Bagging Regressor -----------------------')
# # Bagging Regressor
# bagging = BaggingRegressor(estimator=RandomForestRegressor(), n_estimators=10, random_state=42)
# bagging.fit(X_train_scaled, y_train)
# y_pred_bagging = bagging.predict(X_val_scaled)
# evaluate_model(bagging, X_scaled, y)

# print('---------------------- AdaBoost Regressor ----------------------')
# # AdaBoost Regressor
# adaboost = AdaBoostRegressor(estimator=RandomForestRegressor(), n_estimators=50, random_state=42)
# adaboost.fit(X_train_scaled, y_train)
# y_pred_adaboost = adaboost.predict(X_val_scaled)
# evaluate_model(adaboost, X_scaled, y)

# print('---------------------- Stacking Regressor ----------------------')
# # Stacking Regressor
# estimators = [
#     ('rf', RandomForestRegressor()),
#     ('lr', LinearRegression()),
#     ('svr', SVR())
# ]

# stacking = StackingRegressor(estimators=estimators, final_estimator=RidgeCV())
# stacking.fit(X_train_scaled, y_train)
# y_pred_stacking = stacking.predict(X_val_scaled)
# evaluate_model(stacking, X_scaled, y)

In [ ]:
# Create subplots
fig, axes = plt.subplots(nrows=1, ncols=2, figsize=(18, 6))

# Plot the bar plot to compare the actual and predicted values
axes[0].bar(positive_physical_inv.loc[y_val.index, 'ProductNumber'], y_val, label='Actual', alpha=0.6)
axes[0].bar(positive_physical_inv.loc[y_val.index, 'ProductNumber'], y_pred_rf, label='Predicted', alpha=0.6)
axes[0].set_xlabel('Product Number')
axes[0].set_ylabel('Order Quantity')
axes[0].set_title('Actual vs Predicted Order Quantity')
axes[0].set_ylim(0, 1000000)
axes[0].legend()

# Plot the residual plot
residuals = y_val - y_pred_rf
axes[1].scatter(positive_physical_inv.loc[y_val.index, 'ProductNumber'], residuals, alpha=0.6)
axes[1].axhline(y=0, color='r', linestyle='--')
axes[1].set_xlabel('Product Number')
axes[1].set_ylabel('Residuals')
axes[1].set_title('Residual Plot')

# Show the plots
plt.tight_layout()
plt.show()